In [1]:
import pandas as pd
import sqlite3
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import jieba
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


mpl.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

conn = sqlite3.connect('../sqlite3/qq.db')
cur = conn.cursor()

cur.execute('select * from message')
messages = pd.DataFrame(list(cur.fetchall()))

分词

In [2]:
words = []
def handle(matched):
    return ''
for message in messages.values:
    words.append(re.sub('\[CQ.*?]', handle, message[5]))

tf-idf

In [3]:
# 1000 个特征
vectorizer = CountVectorizer(max_features=1000)
tf_idf_transformer = TfidfTransformer()
tfidf = tf_idf_transformer.fit_transform(vectorizer.fit_transform(words))
print(vectorizer.get_feature_names())

['00', '0点', '0点19', '0点开始', '10', '100', '1002647525', '101', '10瓶', '11', '114514', '11多发言捡到宝的机率更大', '12', '1225290784', '1251316161', '12多发言捡到宝的机率更大', '13', '13多发言捡到宝的机率更大', '14', '14多发言捡到宝的机率更大', '15', '15多发言捡到宝的机率更大', '16', '16多发言捡到宝的机率更大', '17', '17多发言捡到宝的机率更大', '18', '1803945637', '18多发言捡到宝的机率更大', '19', '1939495806', '19多发言捡到宝的机率更大', '1t', '20', '2077', '21', '211', '213531329', '22', '229分', '2313909270', '24', '24盒', '25', '2506984707', '255', '26', '2629557192', '2684319877', '27', '28', '29', '2983', '2983868301', '2984', '2985', '2995', '29元', '2件', '2多发言捡到宝的机率更大', '2瓶', '2袋', '30', '3000', '3002', '3060', '3068640860', '3151292466', '32', '34', '36', '38', '38分', '39', '3件', '415', '418', '423', '433', '436', '44', '446', '457', '461', '49', '49元', '500g', '516', '542', '59', '5d', '5ijyrbnz', '5元', '5多发言捡到宝的机率更大', '5袋', '617670339', '66', '661', '666', '69元', '6元', '6多发言捡到宝的机率更大', '70', '7多发言捡到宝的机率更大', '81', '83', '88', '8元', '8瓶', '90', '91', '912089894', '93', '98', '98

D:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


K-means 聚类

In [4]:
def test_k():
    ks = []
    scores = []
    for i in range(2, 50):
        print(f'k = {i}')
        kmean = KMeans(n_clusters=i, init='k-means++')
        kmean.fit(tfidf)
        km.predict(tfidf)
        s = -km.score(tfidf) / len(words)
        scores.append(s)
        ks.append(i)

    plt.plot(ks, scores, label="error", color="red", linewidth=1)
    plt.xlabel("n_features") # K值
    plt.ylabel("error")
    plt.legend()
    plt.show()
# test_k()

In [5]:
km = KMeans(n_clusters=25, init='k-means++')
km.fit(tfidf)
km.predict(tfidf)
score = -km.score(tfidf) / len(words)
print(score)
messages['cluster'] = km.labels_
messages.sort_values(by='cluster', ascending=False)

0.10148559935436932


,0,1,2,3,4,5,6,cluster
3171,-589544640,288830,850068109,2643785753,0,"[CQ:reply,id=734597933][CQ:at,qq=1812558136] [...",1670914882,24
1501,-1127668253,1341963,45855966,1315896181,0,离谱……,1670811669,24
30880,1681438451,119630,916462960,781432345,0,离谱,1671544731,24
19722,1295527502,118181,916462960,1982972075,0,"[CQ:reply,id=-1283683803][CQ:at,qq=170301048] ...",1671437916,24
23658,1929670826,23757,1002075454,1628898071,0,离谱,1671461404,24
...,...,...,...,...,...,...,...,...
10686,968036037,295355,850068109,2592909875,0,"[CQ:image,file=5341af6655dc78dd7bad89108382364...",1670944926,0
10685,1320435283,295354,850068109,1175215956,0,"[CQ:at,qq=2723898301] 好多写的一个小时多高，去了之后",1670944926,0
10684,-791630864,295353,850068109,3068640860,0,"[CQ:reply,id=-1093883353][CQ:at,qq=2723898301]...",1670944924,0
10683,-1479042202,295352,850068109,1912422440,0,我这边也是,1670944922,0
